In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold

from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score

from tqdm import tqdm

import warnings 
warnings.filterwarnings('ignore')

In [2]:
data = "https://raw.githubusercontent.com/alexeygrigorev/mlbookcamp-code/d12a1a3cdb62c5646282ebe51a4a1e1cf93f47a6/chapter-03-churn-prediction/WA_Fn-UseC_-Telco-Customer-Churn.csv"


In [3]:
# !wget $data

In [4]:
df = pd.read_csv('data-week-3.csv')

df.columns = df.columns.str.lower().str.replace(' ', '_')

categorical_columns = list(df.dtypes[df.dtypes == 'object'].index)

for c in categorical_columns:
    df[c] = df[c].str.lower().str.replace(' ', '_')

df.totalcharges = pd.to_numeric(df.totalcharges, errors='coerce')
df.totalcharges = df.totalcharges.fillna(0)

df.churn = (df.churn == 'yes').astype(int)

In [5]:
df_full_train, df_test = train_test_split(df, test_size=0.2, random_state=1)

In [6]:
numerical = ['tenure', 'monthlycharges', 'totalcharges']

categorical = [
    'gender',
    'seniorcitizen',
    'partner',
    'dependents',
    'phoneservice',
    'multiplelines',
    'internetservice',
    'onlinesecurity',
    'onlinebackup',
    'deviceprotection',
    'techsupport',
    'streamingtv',
    'streamingmovies',
    'contract',
    'paperlessbilling',
    'paymentmethod',
]

In [7]:
def train(df_train, y_train, C=1.0):
    dicts = df_train[categorical + numerical].to_dict(orient='records')
    
    dv = DictVectorizer(sparse=False)
    X_train = dv.fit_transform(dicts)
    
    model = LogisticRegression(C=C, max_iter=1000)
    model.fit(X_train, y_train)
    
    return dv, model

In [8]:
def predict(df, dv, model): 
    dicts = df[categorical + numerical].to_dict(orient='records')
    
    X = dv.transform(dicts)
    y_pred = model.predict_proba(X)[:, 1]
    
    return y_pred

In [9]:
C = 1.0
n_splits = 5

In [10]:
kfold = KFold(n_splits=n_splits, shuffle = True, random_state =1)

scores = []

for train_idx, val_idx in tqdm(kfold.split(df_full_train)):
    df_train = df_full_train.iloc[train_idx]
    df_val = df_full_train.iloc[val_idx]
    
    y_train = df_train.churn.values
    y_val = df_val.churn.values
    
    del df_train['churn']
    del df_val['churn']
    
    dv,model = train(df_train, y_train, C=C)
    y_pred = predict(df_val, dv, model)
    
    auc = roc_auc_score(y_val, y_pred)
    scores.append(auc)
    
print('C=%s %.3f +- %.3f' % (C,np.mean(scores), np.std(scores)))
    

5it [00:01,  3.59it/s]

C=1.0 0.840 +- 0.008


In [11]:
scores

[0.8435839266510193,
 0.8456433750496624,
 0.8311820188641381,
 0.8301724275756219,
 0.8510205202221455]

In [12]:
dv, model = train(df_full_train, df_full_train.churn.values, C=1.0)
y_pred = predict(df_test,dv,model)

auc = roc_auc_score(df_test.churn, y_pred)
auc

0.8572386167896259

In [13]:
## Saving the model

In [14]:
import pickle

In [15]:
output_file = f'model_C={C}.bin'

In [16]:
f_out = open(output_file, 'wb')
pickle.dump((dv,model), f_out)
f_out.close()

In [17]:
!ls -lh *.bin

-rw-r--r-- 1 ramkrishnas ramkrishnas 2.4K Oct 10 11:12 'model_C=1.0.bin'


In [18]:
with open(output_file, 'wb') as f:
    pickle.dump((dv,model), f)

In [19]:
## Load the model

In [1]:
import pickle

In [2]:
input_file = 'model_C=1.0.bin'

In [3]:
with open(input_file, 'rb') as f: 
    dv,model = pickle.load(f)

In [4]:
model

LogisticRegression(max_iter=1000)

In [5]:
customer = {
    'gender': 'female',
    'seniorcitizen': 0,
    'partner': 'yes',
    'dependents': 'no',
    'phoneservice': 'no',
    'multiplelines': 'no_phone_service',
    'internetservice': 'dsl',
    'onlinesecurity': 'no',
    'onlinebackup': 'yes',
    'deviceprotection': 'no',
    'techsupport': 'no',
    'streamingtv': 'no',
    'streamingmovies': 'no',
    'contract': 'month-to-month',
    'paperlessbilling': 'yes',
    'paymentmethod': 'electronic_check',
    'tenure': 1,
    'monthlycharges': 29.85,
    'totalcharges': 29.85
}

In [6]:
X = dv.transform([customer])

In [8]:
y_pred = model.predict_proba(X)[0,1]
y_pred

0.6363584152775206

In [9]:
print('input', customer)
print('output', y_pred)

input {'gender': 'female', 'seniorcitizen': 0, 'partner': 'yes', 'dependents': 'no', 'phoneservice': 'no', 'multiplelines': 'no_phone_service', 'internetservice': 'dsl', 'onlinesecurity': 'no', 'onlinebackup': 'yes', 'deviceprotection': 'no', 'techsupport': 'no', 'streamingtv': 'no', 'streamingmovies': 'no', 'contract': 'month-to-month', 'paperlessbilling': 'yes', 'paymentmethod': 'electronic_check', 'tenure': 1, 'monthlycharges': 29.85, 'totalcharges': 29.85}
output 0.6363584152775206
